In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
testdata_dir = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Fed_test"

## Dasen normalisation results evaluation  
Comparing the betas calculated based on the dasen normalised methylated and unmethylated values obtained from the original R code (dasen() in the wateRmelon package), the centrally implemented pythonic translation of the dasen normalisation and the federated implementation of the pythonic translation of the dasen normalisation

In [10]:
R_dasen = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE66351\\Normalised_Betas.csv" , index_col=0)
Python_dasen = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\QC_Python\\GSE66351_normalised_betas.csv", index_col=0)
Federated_dasen = None

In [3]:
# absolute difference between the beta values
if np.all(R_dasen.index == Python_dasen.index):
    dasen_diff_python = pd.Series.abs((R_dasen) - Python_dasen) # the (* -1) negates the sign difference between python and r calculated
    #regression coefficients
else: print("different probes in the two dataframes")

average_absolute_diff_python_dasen = dasen_diff_python.mean(axis = 1)
print(average_absolute_diff_python_dasen.mean())

0.00012390597584849053


In [6]:
absolute_average_python = Python_dasen.mean(axis=1)
absolute_average_R = R_dasen.mean(axis=1)

In [20]:
print(np.mean((pd.Series.abs((R_dasen - Python_dasen))/R_dasen).max(axis = 1)))
print(np.min(((pd.Series.abs(R_dasen - Python_dasen)/R_dasen).min(axis = 1))))
print(np.max(((pd.Series.abs(R_dasen - Python_dasen)/R_dasen).max(axis = 1))))

0.0008556512048097859
0.0
0.2708994299697678


In [7]:
absolute_av_diff = pd.Series.abs(absolute_average_R - absolute_average_python)
print(absolute_av_diff.max())
print(absolute_av_diff.min())
#Python_dasen.iloc[:,np.argmax(pd.Series.abs(R_dasen.iloc[np.argmax(absolute_av_diff), :]) - pd.Series.abs(Python_dasen.iloc[np.argmax(absolute_av_diff), :]))]


0.002988128437193094
2.853828284798965e-12


In [18]:
# root mean square error between the pythonic/federated beta values and the r beta values (used as ground truth values)
RSEM_probe = np.sqrt(pd.DataFrame.abs((R_dasen - Python_dasen).sum(axis=1)/len(R_dasen))) #for each probe
print("The mean RMSE between the R normalised and the Pyhton normalised data is:", np.sqrt(pd.DataFrame.abs((R_dasen - Python_dasen).sum(axis=1)/len(R_dasen))).mean())
print("The min RMSE between the R normalised and the Pyhton normalised data is:", np.sqrt(pd.DataFrame.abs((R_dasen - Python_dasen).sum(axis=1)/len(R_dasen))).min())
print("The max RMSE between the R normalised and the Pyhton normalised data is:", np.sqrt(pd.DataFrame.abs((R_dasen - Python_dasen).sum(axis=1)/len(R_dasen))).max())

The mean RMSE between the R normalised and the Pyhton normalised data is: 0.0001717427260360975
The min RMSE between the R normalised and the Pyhton normalised data is: 3.467943626858678e-08
The max RMSE between the R normalised and the Pyhton normalised data is: 0.0011221906740329704


## EWAS results evaluation

In [3]:

R_EWAS = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE66351\\EWAS\\Results_small_dataset.csv", index_col=0)
#R_EWAS_test = R_EWAS.iloc[0:21,:]
Python_EWAS = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\QC_Python\\GSE66351full_results_regression_python.csv", header=[0,1], index_col=0)
#Federated_EWAS = pd.read_csv(os.path.join(testdata_dir, "Results_EWASa.csv"), header=[0,1], index_col=0)
#Federated_EWAS_test = Federated_EWAS.iloc[0:21,:]

In [14]:
# check if the same probes are included in the test subset
if np.all(R_EWAS.index == Python_EWAS.index):
    coef_diff_python = pd.Series.abs( R_EWAS["Diagnosis_Beta"] - (-1* Python_EWAS[("Coefficient", "Diagnosis")])) # the (* -1) negates the sign difference between python and r calculated
    #regression coefficients
else: print("different probes in the two dataframes")

average_absolute_diff_python = coef_diff_python.mean()

print(average_absolute_diff_python)

6.046212435902385e-06


In [18]:
# percentage difference
# abs(r - (-1*fed_p)) / r
absolute_relative_diff = pd.Series.abs(R_EWAS["Diagnosis_Beta"] - (-1* Python_EWAS[("Coefficient", "Diagnosis")]))/pd.Series.abs(R_EWAS["Diagnosis_Beta"])
print(absolute_relative_diff.min())
print(absolute_relative_diff.mean())
print(absolute_relative_diff.max())

1.936052645962451e-08
0.011176467686009127
618.3820672863159


In [17]:
test = pd.Series.abs(R_EWAS["Diagnosis_Beta"] - (-1* Python_EWAS[("Coefficient", "Diagnosis")]))
test1 = R_EWAS["Diagnosis_Beta"]

In [ ]:
if np.all(R_EWAS.index == Federated_EWAS.index):
    coef_diff_federated = pd.Series.abs((-1* R_EWAS["Diagnosis_Beta"]) - Federated_EWAS[("Coefficient", "Diagnosis")]) # the (* -1) negates the sign difference between python and r calculated
    #regression coefficients
else: print("different probes in the two dataframes")

average_absolute_diff_federated = coef_diff_federated.mean()

print(average_absolute_diff_federated)

In [23]:
pd.Series.abs(sum(((R_EWAS_test["Diagnosis_Beta"]*-1) - Federated_EWAS_test[("Coefficient", "Diagnosis")]))/len(R_EWAS_test["Diagnosis_Beta"]))

0.0241536102793927

In [13]:
#check root mean square error between the r (ground truth) and python generated regression coefficients
RMSE_python = math.sqrt(pd.Series.abs(sum(((R_EWAS["Diagnosis_Beta"] * -1) - Python_EWAS[("Coefficient", "Diagnosis")]))/len(R_EWAS["Diagnosis_Beta"])))
#RMSE_federated = math.sqrt(pd.Series.abs(sum(((R_EWAS_test["Diagnosis_Beta"]*-1) - Federated_EWAS_test[("Coefficient", "Diagnosis")]))/len(R_EWAS_test["Diagnosis_Beta"])))
print(RMSE_python) #, RMSE_federated)

0.0010689903872974755


## Federated results evaluation  
Test how well it performs with imbalanced splits:  
* Sample size imbalance  
* Class lable imbalance

In [ ]:
# load in the test data